# Data Preprocessing

## Import libraries

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/output/output_dataset.csv')

df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3891,617,16/10/2023,Allemagne,Bluetongue,"Ovins, Bovins",lien raccourci,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,3942,610,17 octobre 2023,Allemagne,Bluetongue,"Bovins, Ovins",lien raccourci,Un animal bovin a été diagnostiqué positif pou...,La maladie a été détectée après les Pays-Bas e...,La détection du virus de la fièvre catarrhale ...
2,code153,https://www.agriland.ie/farming-news/daera-urg...,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4947,770,18 octobre 2023,Irlande du Nord,Bluetongue,"Bovins, Ovins",site officiel,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...
3,code154,https://lc.cx/fnYicq,Erreur,Erreur: Message: unknown error: net::ERR_NAME_...,en,566,42,19 octobre 2023,Allemagne,Bluetongue,"Ovins, Bovins",lien raccourci,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...
4,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,19 octobre 2023,France,Bluetongue,"Ovins, Bovins",lien raccourci,La France détecte des cas de Bluetongue chez l...,Le virus de la fièvre catarrhale ovine (Blueto...,La France a déclaré un état d'urgence après le...


In [3]:
print(df['maladie'])

0                                            Bluetongue
1                                            Bluetongue
2                                            Bluetongue
3                                            Bluetongue
4                                            Bluetongue
5                                            Bluetongue
6                                            Bluetongue
7                  Fièvre catarrhale ovine (Bluetongue)
8                       Dermatose nodulaire contagieuse
9                      Maladie hémorragique épizootique
10               Maladie hémorragique épizootique (MHE)
11                     Maladie hémorragique épizootique
12                      Dermatose nodulaire contagieuse
13                                           Bluetongue
14                Dermatose nodulaire contagieuse (DNC)
15                                           Bluetongue
16                     Maladie hémorragique épizootique
17                                             i

In [4]:
import pandas as pd
import re

# Charger les données
df = pd.read_csv('data/output/output_dataset.csv')

# Dictionnaire de normalisation des maladies
disease_mapping = {
    # Bluetongue / Fièvre catarrhale ovine
    'bluetongue': 'Fièvre catarrhale ovine (Bluetongue)',
    'fièvre catarrhale ovine': 'Fièvre catarrhale ovine (Bluetongue)',
    'fco': 'Fièvre catarrhale ovine (Bluetongue)',
    'langue bleue': 'Fièvre catarrhale ovine (Bluetongue)',
    'bluetongue (inf. par le virus de la)': 'Fièvre catarrhale ovine (Bluetongue)',
    
    # Maladie hémorragique épizootique
    'maladie hémorragique épizootique': 'Maladie hémorragique épizootique (MHE)',
    'mhe': 'Maladie hémorragique épizootique (MHE)',
    'maladie hémorragique épizootique (inf. par le virus de la)': 'Maladie hémorragique épizootique (MHE)',
    
    # Dermatose nodulaire contagieuse
    'dermatose nodulaire contagieuse': 'Dermatose nodulaire contagieuse',
    'dnc': 'Dermatose nodulaire contagieuse',
    'dermatose nodulaire contagieuse (inf. par le virus de la)': 'Dermatose nodulaire contagieuse',
    'lumpy skin disease': 'Dermatose nodulaire contagieuse',
    
    # West Nile / Fièvre du Nil occidental
    'fièvre de west nile': 'Fièvre du Nil occidental (West Nile)',
    'west nile virus': 'Fièvre du Nil occidental (West Nile)',
    'wnv': 'Fièvre du Nil occidental (West Nile)',
    'virus du nil occidental': 'Fièvre du Nil occidental (West Nile)',
    
    # Influenza équine
    'grippe équine': 'Influenza équine',
    'grippe équine (inf. par le virus de la)': 'Influenza équine',
    'equine influenza': 'Influenza équine',
    
    # SARS-CoV-2
    'sars-cov-2': 'SARS-CoV-2 chez les animaux',
    'sars-cov-2 chez les animaux': 'SARS-CoV-2 chez les animaux',
    'sars-cov-2 chez les animaux (inf. par le)': 'SARS-CoV-2 chez les animaux',
    
    # Clavelée et variole caprine
    'clavelée et variole caprine': 'Clavelée et variole caprine',
    'sheep pox': 'Clavelée et variole caprine',
    'goat pox': 'Clavelée et variole caprine',
    
    # Autres maladies
    'nécrose hématopoïétique infectieuse': 'Nécrose hématopoïétique infectieuse',
    'nécrose hématopoïétique infectieuse (inf. par le virus de la)': 'Nécrose hématopoïétique infectieuse',
    'aethina tumida': 'Petit coléoptère des ruches (Aethina tumida)',
    'aethina tumida (petit coléoptère des ruches) (inf. par) (2006-)': 'Petit coléoptère des ruches (Aethina tumida)',
    'peste des petits ruminants': 'Peste des Petits Ruminants (PPR)',
    'ppr': 'Peste des Petits Ruminants (PPR)',
    'péripneumonie contagieuse bovine': 'Péripneumonie Contagieuse Bovine (PPCB)',
    'ppcb': 'Péripneumonie Contagieuse Bovine (PPCB)',
    'fièvre aphteuse': 'Fièvre Aphteuse (Foot And Mouth Disease)',
    'foot and mouth disease': 'Fièvre Aphteuse (Foot And Mouth Disease)',
    'fièvre hémorragique de crimée-congo': 'Fièvre Hémorragique De Crimée-Congo',
}

def normalize_disease_name(disease_name):
    """
    Normalise le nom d'une maladie en utilisant le dictionnaire de mapping
    """
    if pd.isna(disease_name):
        return 'Maladie non spécifiée'
    
    # Convertir en minuscules et nettoyer
    disease_clean = str(disease_name).lower().strip()
    
    # Chercher une correspondance exacte
    if disease_clean in disease_mapping:
        return disease_mapping[disease_clean]
    
    # Chercher une correspondance partielle
    for key, value in disease_mapping.items():
        if key in disease_clean or disease_clean in key:
            return value
    
    # Si aucune correspondance, retourner le nom original avec première lettre en majuscule
    return ' '.join(word.capitalize() for word in disease_name.split())

# Afficher les statistiques AVANT normalisation
print("=" * 60)
print("AVANT NORMALISATION")
print("=" * 60)
print(f"Nombre de maladies uniques : {df['maladie'].nunique()}")
print("\nDistribution des maladies :")
print(df['maladie'].value_counts())

# Appliquer la normalisation DIRECTEMENT sur la colonne 'maladie'
df['maladie'] = df['maladie'].apply(normalize_disease_name)

# Afficher les statistiques APRÈS normalisation
print("\n" + "=" * 60)
print("APRÈS NORMALISATION")
print("=" * 60)
print(f"Nombre de maladies uniques : {df['maladie'].nunique()}")
print("\nDistribution des maladies normalisées :")
print(df['maladie'].value_counts())

# Sauvegarder le fichier avec les maladies normalisées
df.to_csv('data/output/donnees_normalisees.csv', index=False, encoding='utf-8')
print("\n✅ Fichier sauvegardé : data/output/donnees_normalisees.csv")

# Afficher quelques exemples de transformation
print("\n" + "=" * 60)
print("EXEMPLES DE TRANSFORMATIONS")
print("=" * 60)
sample_transformations = df[['code', 'maladie']].head(10)
print(sample_transformations.to_string(index=False))

AVANT NORMALISATION
Nombre de maladies uniques : 21

Distribution des maladies :
maladie
Bluetongue                                                                     15
Dermatose nodulaire contagieuse                                                 6
Maladie hémorragique épizootique                                                6
Fièvre de West Nile                                                             3
Fièvre catarrhale ovine (Bluetongue)                                            3
SARS-CoV-2 chez les animaux (Inf. par le)                                       2
Clavelée et variole caprine                                                     1
Infection à virus West Nile                                                     1
Fièvre hémorragique de Crimée-Congo                                             1
Fièvre aphteuse (Foot and mouth disease)                                        1
Nécrose hématopoïétique infectieuse (Inf. par le virus de la)                   1
Aethina t

In [5]:
df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3891,617,16/10/2023,Allemagne,Fièvre catarrhale ovine (Bluetongue),"Ovins, Bovins",lien raccourci,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,3942,610,17 octobre 2023,Allemagne,Fièvre catarrhale ovine (Bluetongue),"Bovins, Ovins",lien raccourci,Un animal bovin a été diagnostiqué positif pou...,La maladie a été détectée après les Pays-Bas e...,La détection du virus de la fièvre catarrhale ...
2,code153,https://www.agriland.ie/farming-news/daera-urg...,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4947,770,18 octobre 2023,Irlande du Nord,Fièvre catarrhale ovine (Bluetongue),"Bovins, Ovins",site officiel,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...,Le DAERA de l'Irlande du Nord a mis en garde l...
3,code154,https://lc.cx/fnYicq,Erreur,Erreur: Message: unknown error: net::ERR_NAME_...,en,566,42,19 octobre 2023,Allemagne,Fièvre catarrhale ovine (Bluetongue),"Ovins, Bovins",lien raccourci,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...,Le virus de la fièvre catarrhale ovine (Blueto...
4,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,19 octobre 2023,France,Fièvre catarrhale ovine (Bluetongue),"Ovins, Bovins",lien raccourci,La France détecte des cas de Bluetongue chez l...,Le virus de la fièvre catarrhale ovine (Blueto...,La France a déclaré un état d'urgence après le...


In [6]:
df_clean = df[
    ~df['titre'].str.contains('page non trouvée|404|erreur|not found', case=False, na=False) &
    ~df['maladie'].str.contains('inconnu', case=False, na=False) &
    df['maladie'].notna()
]

print(f"Lignes après nettoyage : {len(df_clean)}")
print(f"Lignes supprimées : {len(df) - len(df_clean)}")

# Sauvegarder
df_clean.to_csv('data/output/dataset_nettoye.csv', index=False)
print("✅ Dataset nettoyé sauvegardé !")

Lignes après nettoyage : 45
Lignes supprimées : 5
✅ Dataset nettoyé sauvegardé !


In [7]:
df.columns

Index(['code', 'url', 'titre', 'contenu', 'langue', 'nb_caracteres', 'nb_mots',
       'date_publication', 'lieu', 'maladie', 'animal', 'source_publication',
       'resume_50_mots', 'resume_100_mots', 'resume_150_mots'],
      dtype='object')

In [8]:
import pandas as pd
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
import time
from tqdm import tqdm
import re

# Charger le dataset
df = pd.read_csv('data/output/dataset_nettoye.csv')

translator = GoogleTranslator(source='auto', target='fr')

# Dictionnaire de traduction pour les noms d'animaux
ANIMAUX_DICT = {
    # === Translittérations arabes vers français ===
    'aghnam': 'ovins',
    'agnam': 'ovins',
    'abqar': 'bovins',
    'abkar': 'bovins',
    'maaez': 'caprins',
    'maez': 'caprins',
    'maàz': 'caprins',
    'mashia': 'bétail',  # ماشية (bétail en général)
    'mashiya': 'bétail',
    'dajaj': 'volailles',
    'djaj': 'volailles',
    'khayl': 'équins',
    'jamal': 'camélidés',
    'himaar': 'ânes',
    'tair': 'oiseaux',
    'tayour': 'oiseaux',
    'dewan': 'volailles',
    
    # === Mots arabes en caractères arabes ===
    'أغنام': 'ovins',
    'أبقار': 'bovins',
    'ماعز': 'caprins',
    'ماشية': 'bétail',
    'دجاج': 'volailles',
    'طائر': 'oiseaux',
    'طيور': 'oiseaux',
    'خيل': 'équins',
    'جمال': 'camélidés',
    'حمير': 'ânes',
    'جاموس': 'buffles',
    'أبقار وجاموس': 'bovins et buffles',
    
    # === Mots anglais (au cas où) ===
    'sheep': 'ovins',
    'cattle': 'bovins',
    'goat': 'caprins',
    'goats': 'caprins',
    'poultry': 'volailles',
    'chicken': 'volailles',
    'birds': 'oiseaux',
    'livestock': 'bétail',
    'cows': 'bovins',
    'buffalo': 'buffles',
    
    # === Termes spécifiques ===
    'vaches': 'bovins',
    'moutons': 'ovins',
    'chèvres': 'caprins',
    'poulets': 'volailles',
}

def contient_arabe(texte):
    """Détecte si le texte contient des caractères arabes"""
    if pd.isna(texte) or str(texte).strip() == '':
        return False
    pattern = re.compile(r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]+')
    return bool(pattern.search(str(texte)))

def nettoyer_texte(texte):
    """Nettoie le texte en enlevant les espaces superflus et la ponctuation"""
    if pd.isna(texte):
        return ''
    texte = str(texte).strip()
    # Enlever les espaces multiples
    texte = re.sub(r'\s+', ' ', texte)
    return texte

def traduire_noms_animaux(texte):
    """Traduit les noms d'animaux via le dictionnaire personnalisé"""
    if pd.isna(texte) or str(texte).strip() == '':
        return texte
    
    texte_original = str(texte)
    texte_nettoye = nettoyer_texte(texte_original)
    
    # Séparer par virgules s'il y a plusieurs animaux
    animaux = [a.strip() for a in texte_nettoye.split(',')]
    animaux_traduits = []
    
    for animal in animaux:
        if not animal:
            continue
            
        animal_lower = animal.lower().strip()
        traduit = False
        
        # Chercher une correspondance exacte d'abord
        if animal_lower in ANIMAUX_DICT:
            animaux_traduits.append(ANIMAUX_DICT[animal_lower].capitalize())
            traduit = True
        # Chercher les caractères arabes
        elif animal in ANIMAUX_DICT:
            animaux_traduits.append(ANIMAUX_DICT[animal].capitalize())
            traduit = True
        else:
            # Chercher une correspondance partielle
            for key, value in ANIMAUX_DICT.items():
                if key in animal_lower or animal_lower in key:
                    animaux_traduits.append(value.capitalize())
                    traduit = True
                    break
        
        # Si pas trouvé dans le dictionnaire, essayer de traduire via l'API
        if not traduit:
            try:
                # Traduire si contient de l'arabe
                if contient_arabe(animal):
                    traduction = translator.translate(animal)
                    animaux_traduits.append(traduction.capitalize())
                    time.sleep(0.2)
                else:
                    # Garder tel quel si déjà en français ou non reconnu
                    animaux_traduits.append(animal)
            except Exception as e:
                # En cas d'erreur, garder l'original
                animaux_traduits.append(animal)
    
    # Supprimer les doublons tout en gardant l'ordre
    animaux_uniques = []
    for a in animaux_traduits:
        if a.lower() not in [x.lower() for x in animaux_uniques]:
            animaux_uniques.append(a)
    
    return ', '.join(animaux_uniques)

def est_francais(texte):
    """Détecte si le texte est déjà en français"""
    if pd.isna(texte) or str(texte).strip() == '':
        return True
    
    # Si contient de l'arabe, ce n'est pas français
    if contient_arabe(texte):
        return False
    
    try:
        echantillon = str(texte)[:200].strip()
        if len(echantillon) < 3:
            return True
        langue = detect(echantillon)
        return langue == 'fr'
    except LangDetectException:
        return True
    except Exception:
        return True

def traduire_si_necessaire(texte, nom_colonne=''):
    """Traduit uniquement si ce n'est pas en français"""
    if pd.isna(texte):
        return texte
    
    texte_str = str(texte).strip()
    
    if texte_str == '':
        return texte_str
    
    # Traitement spécial pour la colonne 'animal'
    if nom_colonne.lower() == 'animal':
        return traduire_noms_animaux(texte_str)
    
    # Vérifier si déjà en français
    if est_francais(texte_str):
        return texte_str
    
    try:
        texte_limite = texte_str[:4999]
        traduction = translator.translate(texte_limite)
        time.sleep(0.3)
        return traduction
    
    except Exception as e:
        print(f"\n⚠️ Erreur lors de la traduction de '{nom_colonne}': {e}")
        print(f"   Texte original: {texte_str[:100]}...")
        return texte_str

# ============================================================
# TRADUCTION DES COLONNES
# ============================================================

colonnes_a_traduire = ['titre', 'contenu', 'lieu', 'maladie', 'animal', 
                       'resume_50_mots', 'resume_100_mots', 'resume_150_mots']

print("🚀 Début de la traduction du dataset")
print(f"📁 Fichier source: data/output/dataset_nettoye.csv")
print(f"📊 Nombre de lignes: {len(df)}")
print(f"📋 Colonnes à traiter: {len(colonnes_a_traduire)}\n")

for colonne in colonnes_a_traduire:
    if colonne not in df.columns:
        print(f"⚠️ Colonne '{colonne}' non trouvée, passage à la suivante...")
        continue
    
    print(f"\n{'='*70}")
    print(f"🔄 Traitement de la colonne '{colonne}'")
    print(f"{'='*70}")
    
    # Afficher quelques exemples avant traduction (pour la colonne animal)
    if colonne.lower() == 'animal':
        print(f"\n📋 Exemples AVANT traduction:")
        exemples = df[colonne].dropna().unique()[:8]
        for i, ex in enumerate(exemples, 1):
            print(f"   {i}. {ex}")
    
    # Compter combien nécessitent une traduction
    if colonne.lower() == 'animal':
        nb_a_traduire = len(df[df[colonne].notna()])
    else:
        non_francais = df[colonne].apply(lambda x: not est_francais(x))
        nb_a_traduire = non_francais.sum()
    
    total = len(df)
    
    print(f"\n📊 Statistiques:")
    print(f"   • Total d'entrées: {total}")
    print(f"   • À traiter: {nb_a_traduire}")
    print(f"   • Déjà en français: {total - nb_a_traduire}")
    
    # Détecter les textes avec arabe
    avec_arabe = df[colonne].apply(contient_arabe).sum()
    if avec_arabe > 0:
        print(f"   • Contenant de l'arabe: {avec_arabe}")
    
    if nb_a_traduire > 0:
        print(f"\n⏳ Traduction en cours...")
        df[colonne] = [
            traduire_si_necessaire(texte, colonne)
            for texte in tqdm(df[colonne], desc=f"  {colonne}", 
                            unit="texte", ncols=80)
        ]
        
        # Afficher quelques exemples après traduction (pour la colonne animal)
        if colonne.lower() == 'animal':
            print(f"\n✨ Exemples APRÈS traduction:")
            exemples = df[colonne].dropna().unique()[:8]
            for i, ex in enumerate(exemples, 1):
                print(f"   {i}. {ex}")
        
        print(f"✅ '{colonne}' terminée !")
    else:
        print(f"✓ Aucune traduction nécessaire pour '{colonne}'")

# ============================================================
# SAUVEGARDE
# ============================================================

print(f"\n{'='*70}")
print("💾 Sauvegarde du dataset traduit...")

output_path = 'data/output/dataset_traduit.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ Traduction terminée et sauvegardée dans '{output_path}'!")
print(f"📊 Dataset final: {len(df)} lignes × {len(df.columns)} colonnes")

# ============================================================
# RÉSUMÉ FINAL
# ============================================================

print(f"\n{'='*70}")
print("🐑 RÉSUMÉ - Noms d'animaux traduits:")
print(f"{'='*70}")

if 'animal' in df.columns:
    animaux_uniques = sorted(df['animal'].dropna().unique())
    print(f"\n📋 {len(animaux_uniques)} types d'animaux distincts trouvés:\n")
    
    for i, animal in enumerate(animaux_uniques, 1):
        count = (df['animal'] == animal).sum()
        print(f"   {i:2d}. {animal:<30} ({count} occurrence{'s' if count > 1 else ''})")

print(f"\n{'='*70}")
print("✨ Traitement terminé avec succès !")
print(f"{'='*70}\n")

🚀 Début de la traduction du dataset
📁 Fichier source: data/output/dataset_nettoye.csv
📊 Nombre de lignes: 45
📋 Colonnes à traiter: 8


🔄 Traitement de la colonne 'titre'

📊 Statistiques:
   • Total d'entrées: 45
   • À traiter: 31
   • Déjà en français: 14
   • Contenant de l'arabe: 10

⏳ Traduction en cours...


  titre: 100%|███████████████████████████████| 45/45 [00:46<00:00,  1.03s/texte]


✅ 'titre' terminée !

🔄 Traitement de la colonne 'contenu'

📊 Statistiques:
   • Total d'entrées: 45
   • À traiter: 17
   • Déjà en français: 28
   • Contenant de l'arabe: 10

⏳ Traduction en cours...


  contenu:  56%|████████████████             | 25/45 [00:20<00:13,  1.53texte/s]


⚠️ Erreur lors de la traduction de 'contenu': Request exception can happen due to an api connection error. Please check your connection and try again
   Texte original: الرئيسية أخبار المحافظات تحصين 79 ألفا و222 رأس أغنام وماعز للوقاية من طاعون المجترات الصغيرة بالشرق...


...

✅ 'resume_50_mots' terminée !

🔄 Traitement de la colonne 'resume_100_mots'

📊 Statistiques:
   • Total d'entrées: 45
   • À traiter: 1
   • Déjà en français: 44
   • Contenant de l'arabe: 1

⏳ Traduction en cours...


  resume_100_mots: 100%|█████████████████████| 45/45 [00:00<00:00, 48.64texte/s]


✅ 'resume_100_mots' terminée !

🔄 Traitement de la colonne 'resume_150_mots'

📊 Statistiques:
   • Total d'entrées: 45
   • À traiter: 1
   • Déjà en français: 44
   • Contenant de l'arabe: 1

⏳ Traduction en cours...


  resume_150_mots: 100%|█████████████████████| 45/45 [00:00<00:00, 64.96texte/s]

✅ 'resume_150_mots' terminée !

💾 Sauvegarde du dataset traduit...
✅ Traduction terminée et sauvegardée dans 'data/output/dataset_traduit.csv'!
📊 Dataset final: 45 lignes × 15 colonnes

🐑 RÉSUMÉ - Noms d'animaux traduits:

📋 25 types d'animaux distincts trouvés:

    1. Abeilles (DOMESTIQUE)          (1 occurrence)
    2. Abovins, Douans                (1 occurrence)
    3. Abpars (Bos taurus)            (1 occurrence)
    4. Agnans, Ovins, Doves           (1 occurrence)
    5. Bovins                         (8 occurrences)
    6. Bovins, Buffles                (1 occurrence)
    7. Bovins, Ovins                  (4 occurrences)
    8. Bovins, Ovins, Caprins, Cervidés d'élevage (1 occurrence)
    9. Bovins, Têtes de bétail        (1 occurrence)
   10. Bovins, Élaphe (SAUVAGE), Daim européen (SAUVAGE), Ovins (DOMESTIQUE) (1 occurrence)
   11. Bovins, Équidés                (1 occurrence)
   12. Bétail, Oiseaux                (1 occurrence)
   13. Caprins                        (1 occurr

In [9]:
from datetime import datetime
import re

# Charger le dataset
df = pd.read_csv('data/output/dataset_traduit.csv')

def normaliser_date(date_str):
    """
    Convertit différents formats de date en format jj/mm/aaaa
    """
    if pd.isna(date_str) or str(date_str).strip() == '':
        return date_str
    
    date_str = str(date_str).strip()
    
    # Liste des formats de date possibles
    formats_possibles = [
        '%d/%m/%Y',      # 19/10/2023
        '%d-%m-%Y',      # 19-10-2023
        '%Y-%m-%d',      # 2023-10-19
        '%Y/%m/%d',      # 2023/10/19
        '%d %B %Y',      # 19 octobre 2023
        '%d %b %Y',      # 19 oct 2023
        '%B %d, %Y',     # October 19, 2023
        '%b %d, %Y',     # Oct 19, 2023
        '%d.%m.%Y',      # 19.10.2023
        '%d %m %Y',      # 19 10 2023
    ]
    
    # Dictionnaire pour traduire les mois français en anglais
    mois_fr_en = {
        'janvier': 'january', 'février': 'february', 'mars': 'march',
        'avril': 'april', 'mai': 'may', 'juin': 'june',
        'juillet': 'july', 'août': 'august', 'septembre': 'september',
        'octobre': 'october', 'novembre': 'november', 'décembre': 'december',
        'janv': 'jan', 'févr': 'feb', 'mars': 'mar', 'avr': 'apr',
        'juin': 'jun', 'juil': 'jul', 'sept': 'sep', 'oct': 'oct',
        'nov': 'nov', 'déc': 'dec'
    }
    
    # Remplacer les mois français par anglais
    date_str_lower = date_str.lower()
    for fr, en in mois_fr_en.items():
        if fr in date_str_lower:
            date_str = re.sub(fr, en, date_str_lower, flags=re.IGNORECASE)
            break
    
    # Essayer chaque format
    for fmt in formats_possibles:
        try:
            date_obj = datetime.strptime(date_str, fmt)
            return date_obj.strftime('%d/%m/%Y')
        except ValueError:
            continue
    
    # Si aucun format ne correspond, essayer avec dateutil
    try:
        from dateutil import parser
        date_obj = parser.parse(date_str, dayfirst=True)
        return date_obj.strftime('%d/%m/%Y')
    except:
        # Si tout échoue, retourner la date originale
        print(f"⚠️ Format de date non reconnu: {date_str}")
        return date_str

# Afficher les formats de date actuels
print("📊 Formats de date actuels (10 premiers exemples):")
print("="*60)
for i, date in enumerate(df['date_publication'].head(10), 1):
    print(f"{i:2d}. {date}")

print(f"\n{'='*60}")
print("🔄 Normalisation des dates en cours...")
print(f"{'='*60}\n")

# Normaliser les dates
df['date_publication'] = df['date_publication'].apply(normaliser_date)

# Vérifier les résultats
print("✅ Formats de date après normalisation (10 premiers exemples):")
print("="*60)
for i, date in enumerate(df['date_publication'].head(10), 1):
    print(f"{i:2d}. {date}")

# Compter les formats uniques après normalisation
formats_uniques = df['date_publication'].unique()
print(f"\n{'='*60}")
print(f"📈 Statistiques:")
print(f"   • Nombre total de dates: {len(df)}")
print(f"   • Formats uniques restants: {len(formats_uniques)}")

# Vérifier s'il y a des dates non converties
dates_non_standard = df[~df['date_publication'].str.match(r'^\d{2}/\d{2}/\d{4}$', na=False)]
if len(dates_non_standard) > 0:
    print(f"   ⚠️ Dates non converties: {len(dates_non_standard)}")
    print("\n📋 Exemples de dates non converties:")
    for i, date in enumerate(dates_non_standard['date_publication'].head(5), 1):
        print(f"   {i}. {date}")
else:
    print(f"   ✓ Toutes les dates sont au format jj/mm/aaaa")

# Sauvegarder
print(f"\n{'='*60}")
print("💾 Sauvegarde du dataset avec dates normalisées...")
output_path = 'data/output/dataset.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ Dataset sauvegardé dans '{output_path}'!")
print(f"📊 {len(df)} lignes × {len(df.columns)} colonnes")
print(f"{'='*60}\n")

# Afficher un échantillon aléatoire de dates pour vérification
print("🔍 Échantillon aléatoire de 5 dates pour vérification:")
print("="*60)
echantillon = df.sample(5)
for idx, row in echantillon.iterrows():
    print(f"Code: {row['code']:<10} | Date: {row['date_publication']}")

📊 Formats de date actuels (10 premiers exemples):
 1. 16/10/2023
 2. 17 octobre 2023
 3. 18 octobre 2023
 4. 19 octobre 2023
 5. 19/10/2023
 6. 19 octobre 2023
 7. 17/10/2023
 8. 5 décembre 2025
 9. 19/10/2023
10. 19/10/2023

🔄 Normalisation des dates en cours...

✅ Formats de date après normalisation (10 premiers exemples):
 1. 16/10/2023
 2. 17/10/2023
 3. 18/10/2023
 4. 19/10/2023
 5. 19/10/2023
 6. 19/10/2023
 7. 17/10/2023
 8. 05/12/2025
 9. 19/10/2023
10. 19/10/2023

📈 Statistiques:
   • Nombre total de dates: 45
   • Formats uniques restants: 22
   ✓ Toutes les dates sont au format jj/mm/aaaa

💾 Sauvegarde du dataset avec dates normalisées...
✅ Dataset sauvegardé dans 'data/output/dataset.csv'!
📊 45 lignes × 15 colonnes

🔍 Échantillon aléatoire de 5 dates pour vérification:
Code: code162    | Date: 09/10/2023
Code: code163    | Date: 19/10/2023
Code: code175    | Date: 16/06/2025
Code: code190    | Date: 20/10/2023
Code: code183    | Date: 14/10/2023
